# Question 1 - Dataset Preparation

### Importing all libraries

In [80]:
import pandas as pd
import os
import requests
import gzip
import re
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

### Downloading and extracting dataset file

In [13]:
# Specify path
path = "amazon_reviews_us_Office_Products_v1_00.tsv.gz"

# Check whether the specified
isExist = os.path.exists(path)
if not isExist:
    print("Dataset file Not present")
    # Download the dataset file from given link in the assignment
    
    file_url = "https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"

    # Specify the local file name where you want to save the downloaded file
    local_file_name = "amazon_reviews_us_Office_Products_v1_00.tsv.gz"

    # Send an HTTP GET request to the URL
    response = requests.get(file_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Open the local file in binary write mode and write the content of the response to it
        with open(local_file_name, "wb") as file:
            file.write(response.content)
            print(f"File '{local_file_name}' has been downloaded successfully.")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
        
# Unzip the file
with gzip.open("amazon_reviews_us_Office_Products_v1_00.tsv.gz", 'rb') as gzipped_file, open("amazon_reviews_us_Office_Products_v1_00.tsv", 'wb') as output:
    # Read the compressed content and write it to the output file
    output.write(gzipped_file.read())

False
Not present


### Unzipping the downloaded compressed dataset file

In [18]:
with gzip.open("amazon_reviews_us_Office_Products_v1_00.tsv.gz", 'rb') as gzipped_file, open("amazon_reviews_us_Office_Products_v1_00.tsv", 'wb') as output:
    # Read the compressed content and write it to the output file
    output.write(gzipped_file.read())

### Read and removed malicious rows from the .tsv file

In [3]:
dff1 = pd.read_csv("amazon_reviews_us_Office_Products_v1_00.tsv",sep='\t', on_bad_lines='skip')
dff1.head()

C:\Users\HIMANSHUU\AppData\Local\Temp\ipykernel_20084\3440540034.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dff1 = pd.read_csv("amazon_reviews_us_Office_Products_v1_00.tsv",sep='\t', on_bad_lines='skip')


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,43081963,R18RVCKGH1SSI9,B001BM2MAC,307809868,"Scotch Cushion Wrap 7961, 12 Inches x 100 Feet",Office Products,5,0.0,0.0,N,Y,Five Stars,Great product.,2015-08-31
1,US,10951564,R3L4L6LW1PUOFY,B00DZYEXPQ,75004341,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0.0,1.0,N,Y,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...,2015-08-31
2,US,21143145,R2J8AWXWTDX2TF,B00RTMUHDW,529689027,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0.0,0.0,N,Y,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it.",2015-08-31
3,US,52782374,R1PR37BR7G3M6A,B00D7H8XB6,868449945,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2.0,3.0,N,Y,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...,2015-08-31
4,US,24045652,R3BDDDZMZBZDPU,B001XCWP34,33521401,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0.0,0.0,N,Y,Four Stars,Gorgeous colors and easy to use,2015-08-31


In [4]:
# Keep only the "reviews" and "rating" columns
dff1 = dff1[['star_rating', 'review_headline','review_body']]

### The datatype for star_rating is object as shown below

In [5]:
dff1.dtypes

star_rating        object
review_headline    object
review_body        object
dtype: object

### Changing the datatype of the column to float

In [7]:
# Converting to float datatype
dff1['star_rating'] = pd.to_numeric(dff1['star_rating'], errors='coerce')
dff1.dtypes

star_rating        float64
review_headline     object
review_body         object
dtype: object

### Randomizing the rows and balancing the number of rows for creating a dataset with length 50,000 which for the classification sets

In [9]:
df1 = dff1[dff1['star_rating'] == 1].sample(8333, random_state=42)
df2 = dff1[dff1['star_rating'] == 2].sample(8333, random_state=42)
df3 = dff1[dff1['star_rating'] == 3].sample(8334, random_state=42)
df4 = dff1[dff1['star_rating'] == 4].sample(12500, random_state=42)
df5 = dff1[dff1['star_rating'] == 5].sample(12500, random_state=42)

# Concatenate the DataFrames into a new DataFrame df
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

# Shuffle the rows to ensure randomness
df = df.sample(frac=1, random_state=42)

# Reset the index
df.reset_index(drop=True, inplace=True)

In [10]:
df.head()

,star_rating,review_headline,review_body
0,4.0,Great label printer,The usage is quite low but works great. The fa...
1,2.0,I do like that it is black though I thought it...,This is one of the thinnest most cheaply made ...
2,1.0,Horrible and cheap,Horrible. No instruction manual to put it toge...
3,2.0,Cracked in 5 Months,Lasted only 5 months before cracking. And my ...
4,5.0,Awesome bag!,Always a great bag!! I use this for my college...


# Question 2 - Data Cleaning

### Converting datatype to string and calculating length of review body

In [13]:
# Calculate the length of each review and store it in a new column
df['review_length_before_data_cleaning'] = df['review_body'].str.len()

# Converting to string datatype
df['review_body'] = df['review_body'].astype(str)
df['review_headline'] = df['review_headline'].astype(str)

df.head()

,star_rating,review_headline,review_body,review_length_before
0,4.0,great label printer,the usage is quite low but works great. the fa...,139.0
1,2.0,i do like that it is black though i thought it...,this is one of the thinnest most cheaply made ...,837.0
2,1.0,horrible and cheap,horrible. no instruction manual to put it toge...,78.0
3,2.0,cracked in 5 months,lasted only 5 months before cracking. and my ...,106.0
4,5.0,awesome bag!,always a great bag!! i use this for my college...,100.0


### Applying data cleaning steps - lowercase + html/url removal + non aplhabetical character removal

In [16]:
# Converting all reviews into lowercase
df['review_body'] = df['review_body'].str.lower()
df['review_headline'] = df['review_headline'].str.lower()

# Function to remove HTML tags using regular expressions
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# Function to remove URLs using regular expressions
def remove_urls(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text)

# Clean the 'review_body' column
df['review_body'] = df['review_body'].apply(remove_html_tags)
df['review_body'] = df['review_body'].apply(remove_urls)

# Cleaning the review_headline column
df['review_headline'] = df['review_headline'].apply(remove_html_tags)
df['review_headline'] = df['review_headline'].apply(remove_urls)

# Remove non-alphabetical characters from the "reviews" column
df['review_body'] = df['review_body'].str.replace('[^a-zA-Z\s]', '', regex=True)
df['review_headline'] = df['review_headline'].str.replace('[^a-zA-Z\s]', '', regex=True)
df.head()

,star_rating,review_headline,review_body,review_length_before
0,4.0,great label printer,the usage is quite low but works great the fac...,139.0
1,2.0,i do like that it is black though i thought it...,this is one of the thinnest most cheaply made ...,837.0
2,1.0,horrible and cheap,horrible no instruction manual to put it toget...,78.0
3,2.0,cracked in months,lasted only months before cracking and my ca...,106.0
4,5.0,awesome bag,always a great bag i use this for my college t...,100.0


### Extra space removal and contractions

In [18]:
# Remove extra spaces from the "reviews" column
df['review_body'] = df['review_body'].str.replace('\s+', ' ', regex=True)
df['review_headline'] = df['review_headline'].str.replace('\s+', ' ', regex=True)


## Contractions ##

# Function to expand contractions
def expand_contractions(text):
    return contractions.fix(text)

# Apply the function to the "reviews" column
df['review_body'] = df['review_body'].apply(expand_contractions)
df['review_headline'] = df['review_headline'].apply(expand_contractions)
df.head()

,star_rating,review_headline,review_body,review_length_before
0,4.0,great label printer,the usage is quite low but works great the fac...,139.0
1,2.0,i do like that it is black though i thought it...,this is one of the thinnest most cheaply made ...,837.0
2,1.0,horrible and cheap,horrible no instruction manual to put it toget...,78.0
3,2.0,cracked in months,lasted only months before cracking and my carp...,106.0
4,5.0,awesome bag,always a great bag i use this for my college t...,100.0


### Calculating average length of review_body column

In [21]:
# Calculate the length of each review and store it in a new column
df['review_length_after_data_cleaning'] = df['review_body'].str.len()

# Calculate the average length of reviews
average_length_before_data_cleaning = df['review_length_before_data_cleaning'].mean()

# Calculate the average length of reviews
average_length_after_data_cleaning = df['review_length_after_data_cleaning'].mean()

df.head()

,star_rating,review_headline,review_body,review_length_before,review_length_after
0,4.0,great label printer,the usage is quite low but works great the fac...,139.0,136
1,2.0,i do like that it is black though i thought it...,this is one of the thinnest most cheaply made ...,837.0,828
2,1.0,horrible and cheap,horrible no instruction manual to put it toget...,78.0,75
3,2.0,cracked in months,lasted only months before cracking and my carp...,106.0,99
4,5.0,awesome bag,always a great bag i use this for my college t...,100.0,95


In [22]:
print("The average length before and after is data cleaning : ", average_length_before_data_cleaning, " , ",average_length_after_data_cleaning, " respectively")

The average length before and after is :  333.8094847587807 316.61558  respectively


# Question 3 - Preprocessing

### Stop words removal

In [29]:
# Download NLTK stop words if not already downloaded
nltk.download('stopwords')

# Define a function to remove stop words from a text
def remove_stopwords(text):
    # Tokenize the text
    words = nltk.word_tokenize(text)
    
    # Remove stop words
    filtered_words = [word for word in words if word.lower() not in stopwords.words('english')]
    
    # Join the filtered words back into a sentence
    return ' '.join(filtered_words)

# Apply the function to the "review_body" column
df['review_body'] = df['review_body'].apply(remove_stopwords)

# Print the modified DataFrame
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HIMANSHUU\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


,star_rating,review_headline,review_body,review_length_before,review_length_after,review_length_after_preprocessing
0,4.0,great label printer,usage quite low works great fact symbols well ...,139.0,136,86
1,2.0,i do like that it is black though i thought it...,one thinnest cheaply made door hooks ever seen...,837.0,828,463
2,1.0,horrible and cheap,horrible instruction manual put together looke...,78.0,75,59
3,2.0,cracked in months,lasted months cracking carpet thick looking st...,106.0,99,56
4,5.0,awesome bag,always great bag use college teaching goes ter...,100.0,95,63


### After lemmatization

In [34]:
# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('wordnet')

# Define a function to remove stop words and perform lemmatization
def preprocess_text(text):
    # Tokenize the text
    words = nltk.word_tokenize(text)
    
    # Remove stop words
    filtered_words = [word for word in words if word.lower() not in stopwords.words('english')]
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    
    # Join the lemmatized words back into a sentence
    return ' '.join(lemmatized_words)

# Apply the function to the "review_body" column
df['review_body'] = df['review_body'].apply(preprocess_text)

# Print the modified DataFrame
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HIMANSHUU\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HIMANSHUU\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,star_rating,review_headline,review_body,review_length_before,review_length_after,review_length_after_preprocessing
0,4.0,great label printer,usage quite low work great fact symbol well us...,139.0,136,86
1,2.0,i do like that it is black though i thought it...,one thinnest cheaply made door hook ever seen ...,837.0,828,463
2,1.0,horrible and cheap,horrible instruction manual put together looke...,78.0,75,59
3,2.0,cracked in months,lasted month cracking carpet thick looking str...,106.0,99,56
4,5.0,awesome bag,always great bag use college teaching go terra...,100.0,95,63


In [36]:
# Calculate the length of each review and store it in a new column
df['review_length_after_preprocessing'] = df['review_body'].str.len()

# Calculate the average length of reviews
average_length_after_data_preprocessing = df['review_length_after_preprocessing'].mean()
print("The average length before and after is preprocessing: ", average_length_after_data_cleaning,",", average_length_after_data_preprocessing, " respectively")

The average length before and after is preprocessing:  316.61558 , 196.09036  respectively


# Question 4 - Feature extraction

### TF-IDF feature extraction

In [67]:
# TF-IDF features
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df['review_body'])

df['classes'] = df['star_rating'].apply(lambda x: 1 if x in [1.0, 2.0, 3.0] else 2)
y_tfidf = df['classes']

# Step 4: Split the data into training and testing sets
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, y_tfidf, test_size=0.2, random_state=42)

In [69]:
df.head()

,star_rating,review_headline,review_body,review_length_before,review_length_after,review_length_after_preprocessing,classes
0,4.0,great label printer,usage quite low work great fact symbol well us...,139.0,136,82,2
1,2.0,i do like that it is black though i thought it...,one thinnest cheaply made door hook ever seen ...,837.0,828,458,1
2,1.0,horrible and cheap,horrible instruction manual put together looke...,78.0,75,59,1
3,2.0,cracked in months,lasted month cracking carpet thick looking str...,106.0,99,55,1
4,5.0,awesome bag,always great bag use college teaching go terra...,100.0,95,61,2


### Bag of words feature extraction

In [70]:
# Step 3: Create BoW features
count_vectorizer = CountVectorizer()
X_BoW = count_vectorizer.fit_transform(df['review_body'])
y_BoW = df['classes']  # Replace 'target_column' with the actual name of your target column

# Step 4: Split the data into training and testing sets
X_train_BoW, X_test_BoW, y_train_BoW, y_test_BoW = train_test_split(X_BoW, y_BoW, test_size=0.2, random_state=42)

# Question 5 - Perceptron

### For TFIDF

In [71]:
# Perceptron model for TFIDF
perceptron_model = Perceptron()
perceptron_model.fit(X_train_tfidf, y_train_tfidf)

# Evaluate the model
y_pred_tfidf = perceptron_model.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(y_test_tfidf, y_pred_tfidf)

# Calculate precision, recall, and F1-score for training data
y_train_pred_tfidf = perceptron_model.predict(X_train_tfidf)
precision_train_tfidf = precision_score(y_train_tfidf, y_train_pred_tfidf)
recall_train_tfidf = recall_score(y_train_tfidf, y_train_pred_tfidf)
f1_score_train_tfidf = f1_score(y_train_tfidf, y_train_pred_tfidf)

print(f"Accuracy_tfidf: {accuracy_tfidf:.2f}")
print(f"Precision_tfidf (Training): {precision_train_tfidf:.2f}")
print(f"Recall_tfidf (Training): {recall_train_tfidf:.2f}")
print(f"F1-score_tfidf (Training): {f1_score_train_tfidf:.2f}")

Accuracy_tfidf: 0.74
Precision_tfidf (Training): 0.87
Recall_tfidf (Training): 0.92
F1-score_tfidf (Training): 0.90


### Bag Of Words

In [72]:
# Perceptron model for BoW
perceptron_model = Perceptron()
perceptron_model.fit(X_train_BoW, y_train_BoW)

# Evaluate the model
y_pred_BoW = perceptron_model.predict(X_test_BoW)
accuracy_BoW = accuracy_score(y_test_BoW, y_pred_BoW)

# Calculate precision, recall, and F1-score for training data
y_train_pred_BoW = perceptron_model.predict(X_train_BoW)
precision_train_BoW = precision_score(y_train_BoW, y_train_pred_BoW)
recall_train_BoW = recall_score(y_train_BoW, y_train_pred_BoW)
f1_score_train_BoW = f1_score(y_train_BoW, y_train_pred_BoW)

print(f"Accuracy_BoW: {accuracy_BoW:.2f}")
print(f"Precision_BoW (Training): {precision_train_BoW:.2f}")
print(f"Recall_BoW (Training): {recall_train_BoW:.2f}")
print(f"F1-score_BoW (Training): {f1_score_train_BoW:.2f}")

Accuracy_BoW: 0.75
Precision_BoW (Training): 0.85
Recall_BoW (Training): 0.92
F1-score_BoW (Training): 0.88


In [73]:
print("Bag of words - ",precision_train_BoW,recall_train_BoW,f1_score_train_BoW)
print("TFIDF - ",precision_train_tfidf,recall_train_tfidf,f1_score_train_tfidf)

Bag of words -  0.8543916234282242 0.9151812537552574 0.883742294210081
TFIDF -  0.8709280303030303 0.9209893851391949 0.8952594178915605


# Question 6 - SVM

### TFIDF

In [75]:
# Create an SVM model
svm_model = SVC(kernel='linear')  # You can choose different kernels (e.g., 'linear', 'rbf', etc.) as per your needs

# Train the SVM model on the training data
svm_model.fit(X_train_tfidf, y_train_tfidf)

# Evaluate the SVM model
y_pred_tfidf_svm = svm_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test_tfidf, y_pred_tfidf_svm)

# Calculate precision, recall, and F1-score for training data
y_train_pred_tfidf_svm = svm_model.predict(X_train_tfidf)
precision_train_tfidf_svm = precision_score(y_train_tfidf, y_train_pred_tfidf)
recall_train_tfidf_svm = recall_score(y_train_tfidf, y_train_pred_tfidf)
f1_score_train_tfidf_svm = f1_score(y_train_tfidf, y_train_pred_tfidf)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision (Training): {precision_train_tfidf_svm:.2f}")
print(f"Recall (Training): {recall_train_tfidf_svm:.2f}")
print(f"F1-score (Training): {f1_score_train_tfidf_svm:.2f}")

Accuracy: 0.81
Precision (Training): 0.87
Recall (Training): 0.92
F1-score (Training): 0.90


### Bag of words with SVM

In [78]:
# Create an SVM model
svm_model = SVC(kernel='linear')

# Train the SVM model on the training data
svm_model.fit(X_train_BoW, y_train_BoW)

# Evaluate the SVM model
y_pred_BoW_svm = svm_model.predict(X_test_BoW)
accuracy = accuracy_score(y_test_BoW, y_pred_BoW_svm)

# Calculate precision, recall, and F1-score for training data
y_train_pred_BoW_svm = svm_model.predict(X_train_BoW)
precision_train_BoW_svm = precision_score(y_train_BoW, y_train_pred_BoW)
recall_train_BoW_svm = recall_score(y_train_BoW, y_train_pred_BoW)
f1_score_train_BoW_svm = f1_score(y_train_BoW, y_train_pred_BoW)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision (Training): {precision_train_BoW_svm:.2f}")
print(f"Recall (Training): {recall_train_BoW_svm:.2f}")
print(f"F1-score (Training): {f1_score_train_BoW_svm:.2f}")

Accuracy: 0.79
Precision (Training): 0.85
Recall (Training): 0.92
F1-score (Training): 0.88


In [79]:
print("Bag of words - ",precision_train_BoW_svm,recall_train_BoW_svm,f1_score_train_BoW_svm)
print("TFIDF - ",precision_train_tfidf_svm,recall_train_tfidf_svm,f1_score_train_tfidf_svm)

Bag of words -  0.8543916234282242 0.9151812537552574 0.883742294210081
TFIDF -  0.8709280303030303 0.9209893851391949 0.8952594178915605


# Question 7 - Logistic Regression

### TFIDF

In [81]:
# Create a Logistic Regression model
logistic_regression_model = LogisticRegression()

# Train the Logistic Regression model on the training data
logistic_regression_model.fit(X_train_tfidf, y_train_tfidf)

# Evaluate the Logistic Regression model
y_pred_tfidf_lr = logistic_regression_model.predict(X_test_tfidf)
accuracy_tfidf_lr = accuracy_score(y_test_tfidf, y_pred_tfidf_lr)

# Calculate precision, recall, and F1-score for training data
y_train_pred_tfidf_lr = logistic_regression_model.predict(X_train_tfidf)
precision_train_tfidf_lr = precision_score(y_train_tfidf, y_train_pred_tfidf_lr)
recall_train_tfidf_lr = recall_score(y_train_tfidf, y_train_pred_tfidf_lr)
f1_score_train_tfidf_lr = f1_score(y_train_tfidf, y_train_pred_tfidf_lr)

print(f"Accuracy tfidf_lr: {accuracy_tfidf_lr:.2f}")
print(f"Precision tfidf_lr (Training): {precision_train_tfidf_lr:.2f}")
print(f"Recall tfidf_lr (Training): {recall_train_tfidf_lr:.2f}")
print(f"F1-score tfidf_lr (Training): {f1_score_train_tfidf_lr:.2f}")

Accuracy tfidf_lr: 0.81
Precision tfidf_lr (Training): 0.85
Recall tfidf_lr (Training): 0.86
F1-score tfidf_lr (Training): 0.86


### Bag Of Words

In [82]:
# Step 2: Create a Logistic Regression model
logistic_regression_model = LogisticRegression()

# Step 3: Train the Logistic Regression model on the training data
logistic_regression_model.fit(X_train_BoW, y_train_BoW)

# Step 4: Evaluate the Logistic Regression model
y_pred_BoW_lr = logistic_regression_model.predict(X_test_BoW)
accuracy_BoW_lr = accuracy_score(y_test_BoW, y_pred_BoW_lr)

# Calculate precision, recall, and F1-score for training data
y_train_pred_BoW_lr = logistic_regression_model.predict(X_train_BoW)
precision_train_BoW_lr = precision_score(y_train_BoW, y_train_pred_BoW_lr)
recall_train_BoW_lr = recall_score(y_train_BoW, y_train_pred_BoW_lr)
f1_score_train_BoW_lr = f1_score(y_train_BoW, y_train_pred_BoW_lr)

print(f"Accuracy BoW_lr: {accuracy_BoW_lr:.2f}")
print(f"Precision BoW_lr (Training): {precision_train_BoW_lr:.2f}")
print(f"Recall BoW_lr (Training): {recall_train_BoW_lr:.2f}")
print(f"F1-score BoW_lr (Training): {f1_score_train_BoW_lr:.2f}")

Accuracy BoW_lr: 0.80
Precision BoW_lr (Training): 0.91
Recall BoW_lr (Training): 0.88
F1-score BoW_lr (Training): 0.90


C:\Users\HIMANSHUU\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [83]:
print("Bag of words - ",precision_train_BoW_lr,recall_train_BoW_lr,f1_score_train_BoW_lr)
print("TFIDF - ",precision_train_tfidf_lr,recall_train_tfidf_lr,f1_score_train_tfidf_lr)

Bag of words -  0.9125051802735185 0.8819847786901662 0.8969854363988186
TFIDF -  0.8521717547778606 0.8595533747246145 0.8558466485529826


# Question 8 - Naive bayes

### TFIDF

In [84]:
# Create a Naive Bayes model (Multinomial Naive Bayes is commonly used for text classification)
naive_bayes_model = MultinomialNB()

# Train the Naive Bayes model on the training data
naive_bayes_model.fit(X_train_tfidf, y_train_tfidf)

# Evaluate the Naive Bayes model
y_pred_tfidf_NB = naive_bayes_model.predict(X_test_tfidf)
accuracy_tfidf_NB = accuracy_score(y_test_tfidf, y_pred_tfidf_NB)

# Calculate precision, recall, and F1-score for training data
y_train_pred_tfidf_NB = naive_bayes_model.predict(X_train_tfidf)
precision_train_tfidf_NB = precision_score(y_train_tfidf, y_train_pred_tfidf_NB)
recall_train_tfidf_NB = recall_score(y_train_tfidf, y_train_pred_tfidf_NB)
f1_score_train_tfidf_NB = f1_score(y_train_tfidf, y_train_pred_tfidf_NB)

print(f"accuracy_tfidf_NB: {accuracy_tfidf_NB:.2f}")
print(f"precision_train_tfidf_NB (Training): {precision_train_tfidf_NB:.2f}")
print(f"recall_train_tfidf_NB (Training): {recall_train_tfidf_NB:.2f}")
print(f"f1_score_train_tfidf_NB (Training): {f1_score_train_tfidf_NB:.2f}")

accuracy_tfidf_NB: 0.78
precision_train_tfidf_NB (Training): 0.83
recall_train_tfidf_NB (Training): 0.87
f1_score_train_tfidf_NB (Training): 0.85


### Bag of words

In [85]:
# Create a Naive Bayes model (Multinomial Naive Bayes is commonly used for text classification)
naive_bayes_model = MultinomialNB()

# Train the Naive Bayes model on the training data
naive_bayes_model.fit(X_train_BoW, y_train_BoW)

# Evaluate the Naive Bayes model
y_pred_BoW_NB = naive_bayes_model.predict(X_test_BoW)
accuracy_BoW_NB = accuracy_score(y_test_BoW, y_pred_BoW_NB)

# Calculate precision, recall, and F1-score for training data
y_train_pred_BoW_NB = naive_bayes_model.predict(X_train_BoW)
precision_train_BoW_NB = precision_score(y_train_BoW, y_train_pred_BoW_NB)
recall_train_BoW_NB = recall_score(y_train_BoW, y_train_pred_BoW_NB)
f1_score_train_BoW_NB = f1_score(y_train_BoW, y_train_pred_BoW_NB)

print(f"accuracy_BoW_NB: {accuracy_BoW_NB:.2f}")
print(f"precision_train_BoW_NB (Training): {precision_train_BoW_NB:.2f}")
print(f"recall_train_BoW_NB (Training): {recall_train_BoW_NB:.2f}")
print(f"f1_score_train_BoW_NB (Training): {f1_score_train_BoW_NB:.2f}")

accuracy_BoW_NB: 0.78
precision_train_BoW_NB (Training): 0.85
recall_train_BoW_NB (Training): 0.79
f1_score_train_BoW_NB (Training): 0.82


In [86]:
print("Bag of words - ",precision_train_BoW_NB,recall_train_BoW_NB,f1_score_train_BoW_NB)
print("TFIDF - ",precision_train_tfidf_NB,recall_train_tfidf_NB,f1_score_train_tfidf_NB)

Bag of words -  0.853051996985682 0.793510915281394 0.8222049286640726
TFIDF -  0.8253854599264882 0.8657620668936511 0.8450917621758999


# The Final output for python file

In [93]:
# The average length of characters in review_body before and after data cleaning
print("The average length before data cleaning : ", "{:.2f}".format(average_length_before_data_cleaning))
print("The average length after data cleaning", "{:.2f}".format(average_length_after_data_cleaning), "\n")

# The average length of characters in review_body before and after preprocessing
print("The average length before preprocessing: ", "{:.2f}".format(average_length_after_data_cleaning))
print("The average length after  preprocessing: ","{:.2f}".format(average_length_after_data_preprocessing),"\n")

# perceptron model precision, recall, f-1
print("Bag of words perceptron - ","{:.2f}".format(precision_train_BoW,recall_train_BoW,f1_score_train_BoW))
print("TFIDF  perceptron- ","{:.2f}".format(precision_train_tfidf,recall_train_tfidf,f1_score_train_tfidf),"\n")

# SVM precision, recall, f-1
print("Bag of words SVM- ","{:.2f}".format(precision_train_BoW_svm,recall_train_BoW_svm,f1_score_train_BoW_svm))
print("TFIDF SVM- ","{:.2f}".format(precision_train_tfidf_svm,recall_train_tfidf_svm,f1_score_train_tfidf_svm),"\n")

# LR precision, recall, f-1
print("Bag of words Logistic regression - ","{:.2f}".format(precision_train_BoW_lr,recall_train_BoW_lr,f1_score_train_BoW_lr))
print("TFIDF Logistic regression - ","{:.2f}".format(precision_train_tfidf_lr,recall_train_tfidf_lr,f1_score_train_tfidf_lr),"\n")

# Naive bayes precision, recall, f-1
print("Bag of words Naive bayes - ","{:.2f}".format(precision_train_BoW_NB,recall_train_BoW_NB,f1_score_train_BoW_NB))
print("TFIDF Naive bayes - ","{:.2f}".format(precision_train_tfidf_NB,recall_train_tfidf_NB,f1_score_train_tfidf_NB),"\n")

The average length before data cleaning :  333.81
The average length after data cleaning 316.62 

The average length before preprocessing:  316.62
The average length after  preprocessing:  196.09 

Bag of words perceptron -  0.85
TFIDF  perceptron-  0.87 

Bag of words SVM-  0.85
TFIDF SVM-  0.87 

Bag of words Logistic regression -  0.91
TFIDF Logistic regression -  0.85 

Bag of words Naive bayes -  0.85
TFIDF Naive bayes -  0.83 

